In [ ]:
# make sure packages are installed
import subprocess
def install_package(package_name):
    subprocess.check_call(["pip", "install", package_name])

install_package("numpy")
install_package("pandas")
install_package("matplotlib")
install_package("ipympl")
install_package("ipywidgets")
install_package("mplcursors")

In [ ]:
%matplotlib widget

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as mplc
import ipywidgets as widgets
import mplcursors as mplc

filename_snapshot = "Dataset/snapshots_arabia_t_5040.csv"
filename_overview = "Dataset/aoe_data.csv"
filepath_inputs = "Dataset/inputs/"

In [ ]:
data        = pd.read_csv(filename_snapshot)
print("Loaded data")

overview    = pd.read_csv(filename_overview)
print("Loaded overview", )

In [ ]:
# Player town center spawn correlation

map_width = 120

# Interactive parameters
current_angle = 0
current_radius = 10
current_distance = 40

figure = plt.figure(figsize=(12, 9))
plt.title("Player spawn position")

data_p1_filtered = [[], []]
data_p2_matching = [[], []]

data_filtered = overview.loc[
	(overview["p1_xpos"] < 40) & 
	(overview["p1_ypos"] > 50) &
	(overview["p1_ypos"] < 70)]

global matchIDs
matchIDs = data_filtered["match_id"]

# Plots
plot_p1 = plt.plot([], [], "o", alpha=0.03, color="blue")
plot_p2 = plt.plot([], [], "o", alpha=0.03, color="red")

# For viewing individual games
plot_p1_town = plt.plot([], [], "o", markersize=10, alpha=1.0, color="blue")
plot_p2_town = plt.plot([], [], "o", markersize=10, alpha=1.0, color="red")
plot_p1_buildings = plt.plot([], [], "o", markersize=5, alpha=0.8, color="blue")
plot_p2_buildings = plt.plot([], [], "o", markersize=5, alpha=0.8, color="red")

# Widgets
widget_angle = widgets.FloatSlider(description="Angle", value=current_angle, min=0.0, max=math.pi * 2)
widget_radius = widgets.FloatSlider(description="Radius", value=current_radius, min=0.0, max=40)
widget_distance = widgets.FloatSlider(description="Distance", value=current_distance, min=0.0, max=map_width / 2)

# Functional variables
global gameSelected
gameSelected = False
global selectorRef

# When selecting a game via widget
def select_game(index, x, y, selector):
	#print("select")
	global matchIDs
	global selectorRef
	global gameSelected
	global data_filtered
	selectorRef = selector

	inputData = pd.read_csv(filepath_inputs + str(matchIDs.iloc[index]) + "_inputs.csv")
	input_filtered_p1 = inputData.loc[(inputData["type"] == "Build") & (inputData["player"]=="p1")]
	input_filtered_p2 = inputData.loc[(inputData["type"] == "Build") & (inputData["player"]=="p2")]

	selector.annotation.set_text(	"Game: " + str(matchIDs.iloc[index]) + "\n" +
									"Player 1 buildings: " + str(len(np.array(input_filtered_p1))) + "\n" +
									"Player 2 buildings: " + str(len(np.array(input_filtered_p2))))

	plot_p1_town[0].set_xdata([data_filtered["p1_xpos"].iloc[index]])
	plot_p1_town[0].set_ydata([data_filtered["p1_ypos"].iloc[index]])
	plot_p2_town[0].set_xdata([data_filtered["p2_xpos"].iloc[index]])
	plot_p2_town[0].set_ydata([data_filtered["p2_ypos"].iloc[index]])

	plot_p1_buildings[0].set_xdata(input_filtered_p1["x_pos"])
	plot_p1_buildings[0].set_ydata(input_filtered_p1["y_pos"])
	plot_p2_buildings[0].set_xdata(input_filtered_p2["x_pos"])
	plot_p2_buildings[0].set_ydata(input_filtered_p2["y_pos"])
	gameSelected = True
	# Update the other plots
	v4_draw_with_globals()

# When deselcting by clicking anywhere
def deselect_game(event):
	#print("deselect")
	global selectorRef
	global gameSelected
	if not gameSelected:
		plot_p1_buildings[0].set_xdata([])
		plot_p1_buildings[0].set_ydata([])
		plot_p2_buildings[0].set_xdata([])
		plot_p2_buildings[0].set_ydata([])
		plot_p1_town[0].set_xdata([])
		plot_p1_town[0].set_ydata([])
		plot_p2_town[0].set_xdata([])
		plot_p2_town[0].set_ydata([])
		selectorRef.annotation.set_text("")
		v4_draw_with_globals()
	gameSelected = False

def v4_draw(angle, radius, distance):
	#print("drawCall")
	global gameSelected
	global matchIDs
	position = (
		map_width / 2 + distance * math.sin(angle),
		map_width / 2 + distance * math.cos(angle)
	)
	radius_squared = radius * radius
	#print(position)

	global data_filtered
	data_filtered = overview.loc[
		(overview["p1_xpos"] - position[0]) * (overview["p1_xpos"] - position[0]) + 
		(overview["p1_ypos"] - position[1]) * (overview["p1_ypos"] - position[1]) < radius_squared
	]

	matchIDs = data_filtered["match_id"]

	if gameSelected:
		plot_p1[0].set_xdata([])
		plot_p1[0].set_ydata([])
		plot_p2[0].set_xdata([])
		plot_p2[0].set_ydata([])
	else:
		plot_p1[0].set_xdata(data_filtered["p1_xpos"])
		plot_p1[0].set_ydata(data_filtered["p1_ypos"])
		plot_p2[0].set_xdata(data_filtered["p2_xpos"])
		plot_p2[0].set_ydata(data_filtered["p2_ypos"])

def v4_draw_with_globals():
	global current_angle
	global current_radius
	global current_distance
	v4_draw(current_angle, current_radius, current_distance)

def v4_listener_angle(change):
	deselect_game(0)
	global current_angle
	current_angle = change.new
	v4_draw_with_globals()

def v4_listener_radius(change):
	deselect_game(0)
	global current_radius
	current_radius = change.new
	v4_draw_with_globals()

def v4_listener_distance(change):
	deselect_game(0)
	global current_distance
	current_distance = change.new
	v4_draw_with_globals()

widget_angle.observe(v4_listener_angle, names='value')
widget_radius.observe(v4_listener_radius, names='value')
widget_distance.observe(v4_listener_distance, names='value')

v4_draw_with_globals()

# Add cursor listener
global cursors
cursors = mplc.cursor(plot_p1)
figure.canvas.mpl_connect("button_press_event", deselect_game)
cursors.connect("add", lambda sel:
			select_game(sel.index, sel.target[0], sel.target[1], sel)
			)


plt.xlim(0, map_width)
plt.ylim(0, map_width)

display(widget_angle)
display(widget_radius)
display(widget_distance)
plt.tight_layout()
plt.show()

# layout = widgets.AppLayout(
#     left_sidebar = widgets.VBox(children=[widget_angle, widget_distance, widget_radius]),
#     center = figure.canvas
# )